In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [2]:
import yaml
with open('../config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:

ENDPOINT = config["custom_vision"]["endpoint"];
training_key = config["custom_vision"]["training_key"]
prediction_key = config["custom_vision"]["prediction_key"]
prediction_resource_id = config["custom_vision"]["prediction_resource_id"]

In [5]:
publish_iteration_name = "FireNet"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [7]:
project = trainer.get_project(config["project_id"]);

## Upload and tag images

In [8]:
import pandas as pd
from pathlib import Path

firenet_path = Path(config["firenet_path"])
images_path = firenet_path / "Images"

local_path = Path("../data")
df = pd.read_pickle(local_path / "object_detection.pkl")
df.head()

,filename,tag,left,top,width,height,truncated,difficult
0,0000.jpg,Fire_Extinguisher,0.172526,0.497070,0.322917,0.455078,0,0
1,0000.jpg,Fire_Suppression_Signage,0.322266,0.377441,0.128255,0.090332,0,0
2,0001.jpg,Fire_Blanket,0.236613,0.544428,0.298193,0.298005,0,0
3,0001.jpg,Fire_Extinguisher,0.579317,0.637613,0.367805,0.336596,0,0
4,0001.jpg,Fire_Suppression_Signage,0.380857,0.027673,0.326640,0.298005,0,0


### Add tags to the project

In [9]:
tags = {}
for az_tag in trainer.get_tags(project.id):
    tags[az_tag.name] = az_tag
for tag_name in df.tag.unique():
    if tag_name not in tags:
        tags[tag_name] = trainer.create_tag(project.id, tag_name)
len(tags)

8

### Adding images

In [14]:
from tqdm import tqdm

start_at = 350

tagged_images_with_regions = []

print("Loading images and labels...")

images = df.filename.unique()
i = 0
for file_name in tqdm(images):
    i += 1
    if (i <= start_at):
        continue

    rows = df.where(df.filename == file_name).dropna()
    regions = []
    for _, row in rows.iterrows():
        regions.append(
            Region(
                tag_id=tags[row["tag"]].id,
                left=row["left"],
                top=row["top"],
                width=row["width"],
                height=row["height"],
            )
        )

    with open(images_path / file_name, mode="rb") as image_contents:
        tagged_images_with_regions.append(
            ImageFileCreateEntry(
                name=file_name, contents=image_contents.read(), regions=regions
            )
        )

    if i % 50 == 0:
        print(f"Uploading image batch {i} ...")
        upload_result = trainer.create_images_from_files(
            project.id, ImageFileCreateBatch(images=tagged_images_with_regions)
        )
        tagged_images_with_regions = []
        if not upload_result.is_batch_successful:
            errors = False
            for image in upload_result.images:
                if image.status != "OK" and image.status != "OKDuplicate":
                    errors = True
                    print(f"Image {image.source_url}: {image.status}")
            if errors:
                print(f"Image batch {i} upload failed.")
                break

if len(tagged_images_with_regions) > 0:
    print(f"Uploading latest batch {i}...")
    upload_result = trainer.create_images_from_files(
        project.id, ImageFileCreateBatch(images=tagged_images_with_regions)
    )
    tagged_images_with_regions = []
    if not upload_result.is_batch_successful:
        errors = False
        for image in upload_result.images:
            if image.status != "OK" and image.status != "OKDuplicate":
                errors = True
                print(f"Image {image.source_url}: {image.status}")
        if errors:
            print(f"Latest Image batch upload failed.")


Loading images and labels...


 25%|██▍       | 362/1449 [00:00<00:00, 3402.71it/s]

Uploading image batch 400 ...


 28%|██▊       | 405/1449 [00:15<00:53, 19.46it/s]

Uploading image batch 450 ...
Uploading image batch 500 ...


...

 93%|█████████▎| 1342/1449 [04:10<00:11,  9.31it/s]

Uploading image batch 1350 ...


 96%|█████████▋| 1395/1449 [04:26<00:04, 13.43it/s]

Uploading image batch 1400 ...


100%|██████████| 1449/1449 [04:40<00:00,  5.17it/s]
